In [1]:
import pywebio
import pandas as pd
import json
from dicts import fips
from functions import get_districts, calculate_center
from model import create_models, predict_2020, predict_user_cd
from mapping import mapping
from cd_gui import gui_cd
import plotly.express as px
from pywebio.input import *
from pywebio.output import *

In [7]:
def website():

    state_fips = fips()
    state_names = list(state_fips)

    # data = get_final_data()
    data = pd.read_csv('final_data.csv')
    data = data.dropna(axis=0)

    rf_dem, rf_rep, scaler = create_models(data)
    
    while True:
        clear()
        choice = radio("Choose a congressional district, make your own, or just look at the whole map!", options=['Select a state and district', 'Make my own district', 'Just show me the entire country!'], required=True)

        if choice == 'Select a state and district':
            pred_2020 = predict_2020(data, rf_dem, rf_rep, scaler)
            state = select('Select a state: ', options=state_names)

            if state:
                districts = get_districts(pred_2020, state)
                district = select('Select a congressional district: ', options=districts, required=True)
                if district:
                    put_text(f'You selected {state} Congressional District {district}!')

                    predictions = predict_2020(data, rf_dem, rf_rep, scaler)
                    predictions['State FIPS'] = predictions['State FIPS'].astype(str).str.zfill(2)
                    predictions['District'] = predictions['District'].astype(str).str.zfill(2)
                    predictions['FIPS_CD'] = predictions['State FIPS'] + predictions['District']
                    predictions['Dem Advantage'] = predictions['Predicted Democratic %'] - predictions['Predicted Republican %']

                    file_path = 'gz_2010_us_500_11_5m.json'
                    with open(file_path) as file:
                        geo_data = json.load(file)

                    for feature in geo_data["features"]:
                        if feature["properties"]["CD"] == "00":
                            feature["properties"]["CD"] = "01"
                        state_cd = feature["properties"]["STATE"] + feature["properties"]["CD"]
                        feature["id"] = state_cd
                    
                    new_state = state_fips.get(state, "Unknown")
                    state = new_state
                    district = str(district).zfill(2)
                    zoom_level = 6
                    center_lat, center_lon = 0, 0
                    found = False
                    for feature in geo_data["features"]:
                        if feature["properties"]["STATE"] == state and feature["properties"]["CD"] == district:
                            coords = feature["geometry"]["coordinates"][0]
                            center_lat, center_lon = calculate_center(coords)
                            found = True
                            break
                    if not found:
                        put_text("Sorry, the JSON we used for mapping is behind a congressional map so we can't show you that one unfortunately.")
                        try_again = radio("Would you like to try another option?", options=['Yes', 'No'], required=True)
                        if try_again == 'Yes':
                            continue

                    fig = px.choropleth_mapbox(predictions, geojson=geo_data, locations='FIPS_CD', color='Dem Advantage',
                                            color_continuous_scale="RdBu",
                                            range_color=(-50, 50),
                                            mapbox_style="carto-positron",
                                            color_continuous_midpoint=0,
                                            zoom=zoom_level,
                                            center={"lat": center_lat, "lon": center_lon},
                                            opacity=0.7,
                                            labels={'Dem Advantage': 'Predicted Dem Advantage %'},
                                            hover_data=['State', 'District', 'Predicted Winner', 'Predicted Democratic %', 'Predicted Republican %'],
                                            )
                    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
                    html = fig.to_html(include_plotlyjs="require", full_html=False)
                    put_html(html)

        elif choice == 'Make my own district':
            user_cd = gui_cd()
            user_cd_values = list(user_cd.values())
            dem, rep, winner = predict_user_cd(user_cd_values, rf_dem, rf_rep, scaler)
            if user_cd:
                table_data = [['Demographic Feature', 'Percentage']]
                table_data.extend([[key, str(value)] for key, value in user_cd.items()])
                put_text(f"Your congressional district will be split: {dem}% Democratic and {rep}% Republican. The {winner} Party wins!")
                put_text("As a reminder, here was your congressional district's demographic profile:")
                put_table(table_data)

        elif choice == 'Just show me the entire country!':
            put_text("Here's a map of my predictions for the 2020 elections! Feel free to look around.")
            predictions = predict_2020(data, rf_dem, rf_rep, scaler)

            predictions['State FIPS'] = predictions['State FIPS'].astype(str).str.zfill(2)
            predictions['District'] = predictions['District'].astype(str).str.zfill(2)
            predictions['FIPS_CD'] = predictions['State FIPS'] + predictions['District']
            predictions['Dem Advantage'] = predictions['Predicted Democratic %'] - predictions['Predicted Republican %']

            file_path = 'gz_2010_us_500_11_5m.json'
            with open(file_path) as file:
                geo_data = json.load(file)

            for feature in geo_data["features"]:
                if feature["properties"]["CD"] == "00":
                    feature["properties"]["CD"] = "01"
                state_cd = feature["properties"]["STATE"] + feature["properties"]["CD"]
                feature["id"] = state_cd

            zoom_level = 3
            center_lat = 37.0902
            center_lon = -95.7129

            fig = px.choropleth_mapbox(predictions, geojson=geo_data, locations='FIPS_CD', color='Dem Advantage',
                                color_continuous_scale="RdBu",
                                range_color=(-50, 50),
                                mapbox_style="carto-positron",
                                color_continuous_midpoint=0,
                                zoom=zoom_level,
                                center={"lat": center_lat, "lon": center_lon},
                                opacity=0.7,
                                labels={'Dem Advantage': 'Predicted Dem Advantage %'},
                                hover_data=['State', 'District', 'Predicted Winner', 'Predicted Democratic %', 'Predicted Republican %'],
                                )
            fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

            # fig = mapping(pred_2020)
            html = fig.to_html(include_plotlyjs="require", full_html=False)
            put_html(html)

        try_again = radio("Would you like to try another option?", options=['Yes', 'No'], required=True)
        if try_again == 'No':
            break 

if __name__ == "__main__":
    website()